In [1]:
import numpy as np
import torch
from models import *
from main import *
from configs import cfg
from nltk.translate import bleu_score as bs
import time

## Test Text Generation

In [2]:
cfg['gen_temp'] = 0.4
fname = 'reviews/reviews_gru_04_2.txt'

In [3]:
test_data = np.load('/datasets/home/32/032/sugao/CSE190/pa4/test.npy')

In [4]:
df = load_data('/datasets/cs190f-public/Beeradvocate_TestOriginal.csv')
df.dropna(subset=['review/overall', 'beer/style'],how='any',inplace = True)

In [5]:
N = 3000
start = len(df) % N
cfg['batch_size'] = start
model = load_model(BeerGruMind, './models/2-4_5-5/e4_c10050_ls0.369', cfg)

with torch.no_grad():
    all_txt = generate(model, test_data[0:start], cfg)
    li = tensor2strlist(all_txt)
    save_to_file(li, fname)

cfg['batch_size'] = N
model = load_model(BeerGruMind, './models/2-4_5-5/e4_c10050_ls0.369', cfg)
scores = []

for i_batch in range(start, len(test_data), N):
    with torch.no_grad():
        all_txt = generate(model, test_data[i_batch:i_batch + N], cfg)
        li = tensor2strlist(all_txt)        
        score = calc_score(df, [x for x in range(i_batch, i_batch + 10)], li[0:10])
        scores.append(score)
        save_to_file(li, fname)
        print(i_batch, score)

/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


2323 0.34585320245797474
5323 0.3585100648738001
8323 0.34545670746545665
11323 0.3306163108499099
14323 0.3521405160416874
17323 0.3580519198284543
20323 0.3534613792932011


/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


23323 0.06400215297311797
26323 0.3525024286632164
29323 0.3504464120193966
32323 0.35164590386832956
35323 0.05427654767829393
38323 0.35108168497197656
41323 0.35049047393162946
44323 0.057887502588093075
47323 0.05372482865862965
50323 0.349213357024101
53323 0.34329452398451965
56323 0.34925814232249186
59323 0.3576393377107947
62323 0.351870296748681
65323 0.05401312590942165
68323 0.3567309246409057
71323 0.3427229872889962
74323 0.334826009241875
77323 0.3364501178951564
80323 0.34611880091489405
83323 0.3511102649886279
86323 0.36067814893280986
89323 0.3716686915517356
92323 0.35782241396102615
95323 0.3532318736990002
98323 0.3521405160416874
101323 0.05505259776034605
104323 0.3328209054797223
107323 0.33967306119047985
110323 0.05383656606411098
113323 0.3520052767479625
116323 0.3558166823830502
119323 0.3347193646115962
122323 0.3760603093086394
125323 0.3460744636564537
128323 0.3467425211960829
131323 0.3471464676049615
134323 0.35445005868763624
137323 0.34281060662052

In [6]:
np.median(scores)

0.3471464676049615

## Validation

In [2]:
training_data = np.load('/datasets/home/32/032/sugao/CSE190/pa4/train.npy')

In [3]:
valid_indices = np.load('./models/2-4_5-5/val.npy')

In [4]:
df = load_data('/datasets/cs190f-public/BeerAdvocateDataset/BeerAdvocate_Train.csv')
df.dropna(subset=['review/text'],how='any',inplace = True)

In [6]:
scores = []
n = 10

cfg['batch_size'] = 10
model = load_model(BeerGruMind, './models/2-4_5-5/e4_c10050_ls0.369', cfg)

sm = bs.SmoothingFunction()
for i_batch in range(1000, 2000, n):
    meta = np.empty((n, 2))
    data = training_data[valid_indices[i_batch:i_batch + n]]
    for j in range(n):
        meta[j, 0] = data[j][0]
        meta[j, 1] = data[j][1]
        
    all_txt = generate(model, meta, cfg)
    li = tensor2strlist(all_txt)
    score = calc_score(df, valid_indices[i_batch:i_batch + n], li, sm.method0)
    scores.append(score)
    print(score)
    print(df['review/text'].values[valid_indices[i_batch:i_batch + n][0]])
    print(' '.join(li[0]))

print(np.median(scores))


/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.33692603386592873
22oz. bomber that has sat patiently in the DBF for far too long. It is time...		Poured into my Duvel tulip glass, this beauty of a beer is the essence of darkness. Inky, darkest-of-night blackness. Sucking the light in, perhaps. A toffee-colored layer of thick, long-lasting, and lace-producing head remains perched atop the ink below it. Aromas are heavenly...a mixture of milk chocolate, espresso, caramel, roasted malt. The flavor carries all of this as well. Splashes of chocolate and caramel initially, the middle sees some of the chewy grains, with a co-op of ABV and choco-bitterness at the end (what a finish!). Surprisingly, it was slightly on the less-than-big side in the mouthfeel. Still, this is one mammothly drinkable beer. A nice nightcap to a busy day.
Poured from the bottle into a stout is a reddish bottle into a standard on tap at a mind of the beer was a very nice stout in the glass. Color with a thick and lacing on the beer pours a nice light brown head t

/opt/conda/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.06429917630658798
A good offering, but not Old Baldy's best. According to the staff, the batch I sampled was one of the lighter ABV batches in recent times. It had a slightly off nose that was hard to identify. I finally concluded that it was of perhaps currants or raisins. The pour was perfectly opaque black with no head whatsoever. Despite this, there was some ring lacing throughout. No light passes through this brew. Some roasted sense to the palate; very light bitterness in the back. A bit watery for the style.
12 oz bottle into a creamy amber with a thin colored brown color with a slight head that little good brown around the glass, and of spicy head. The head is a bit of sticky lacing and leaves a light amber color with a slight smooth and lacing. Aroma is sweet and caramel malts with a bit of flavor and light malt and coffee and alcohol carbonation leaves a creamy malt and a bit of sweetness and a little light caramel and chocolate and a bit of carbonation. 		Mouthfeel is cont

0.34863374779499706
Had this beer at Papagos. It was served in a Weizen glass.		Beer is a clear orange golden color with a one inch creamy head that faded but left lots of lacing.		First thing that hits you is vanilla! Followed y orange and a bit of wheat on the nose. Reminded me of a cream sicle just like it was advertised.		Taste was a rush of orange and vanilla with a nice malty wheaty finish with just a slight sweet finish.		Mouthfeel was vey smooth and light.		Overall I really enjoyed this beer even though I am not a big fruit beer fan. However I can see this being a real thirst quencher in the hot AZ sun although it was 45 degrees out when I sampled this one. Definitely a beer you need to try as it is unique among craft beers. Way to go Papago!
Poured from a bottle into a clear brown with a slightly cloudy fruit and retention and a thin color with a slight clear color with a bit of lacing on the glass. The nose is sweet and a bit of a smokey malt, with a touch of roasted malt fla

...

0.370811098804299
OKAY so its macro-swill, but its the low cal version of the best macro-swill available. High Life Light is thin and watery but actually sort of has a flavor. Well okay, so its isn't so much a flavor as it is a lack of bad taste. In fact its more like a lack of taste. F+ beer. I've had worse, much worse in fact but I'm not proud of it.
Pours a clear color with a thin color with a slight head that leaves light aroma is a bit of citrus and carbonation, but not the beer is a little balanced and creamy sugary, but the beer is a little sweet and sweet malt and some bitterness that is a bit of the malt drinkability here and the finish was medium body that is some light flavor that is light to be a bad beer profile. The taste is a bit of burnt coffee and some caramel, malty and caramel and light caramel and a small sweetness that leaves a bit of some caramel and pleasant in the taste with a little malty and coffee. A bit of sweet malts, and a slight caramel and a bit of caram

0.3460301547811259
Had this at the Flying Saucer in Charlotte, NC for one of their rare beer Tap Nights!		This incredible beer is a maple barrel aged version of the already awesome Dirty Bastard! It pours a brick red color with maple, caramel and pine in the nose. The complex flavor is of rich maple syrup that reminds you of gorging yourself on french toast at your grandmother's house when you were five, some oak notes, bittersweet chocolate, a hint of coffee, figs, and dates, with aromatics to match. It all combines to create a well-rounded, rich and decadent beer. This rare treat is usually only available at the brewery.
Poured from a 12oz bottle into a still amount of cloudy brown with a bit of some malty and clean color with a dark brown with a thin white head that retention and some clear amber in the head retention and lacing that leaves a minimal head in the glass. The head is a little light brown head. The head retention as well. The aroma is creamy with a slight caramel and sw

0.05337886772360096
A- Pinkish, reddish, grapefruit colored murky amber. Its an unusual color. Nice fluffy head but poor retention. Im using a big Chimay goblet so it takes a special beer to hold a big head in such a wide glass.		S- Nice smell. Right in the middle of all the dark fruity strong dark Belgians and the light juicy fruitiness of a Tripel. Lots of Red Delicious apples, sweet cherries, strawberries, pears, peach, kiwi, lots of chalky smelling yeasts that are typical to Tripels. Its like strawberry/kiwi yogurt at times. 		T- Lots more chalky yeasts right up front that give a dryness throughout that fights all the sweet, juicy fruits. Lots more sweet apples, strawberries, sweet cherries maybe a little bubblegum from the yeast. Spiciness is moderate and towards the end. Its a little like cinnamon apples.		M- Dry, medium bodied with very fluffy carbonation.		D- A touch above a session ale. I think even if the ABV were low it still would be far from sessionable. Its a bit rich and

0.05286314346838192
I thought Id hold out and sample Full Sails best selling new beer on draught at the Hood River brewery where its produced. But as I sat down at the bar in the newly remodeled taproom I noticed a small bucket full of Session bottle caps and the bartender informed me that the beer is only served in the 11-oz. stubby bottles, never on draught. They dont even pour it in a glass for you, but simply crack the bottle and hand it to you. 		Its been a mighty long time since Ive been served a beer like that. I ordinarily visit brewpubs to drink draught beers or at the very least, get a bottled beer served in a glass. So I had to smile at being caught off guard by the quaint old-school service. It seems Full Sail has taken the whole working mans session lager thing quite seriously. And it somehow seems appropriate. 		Bear with me because Ive never, ever consumed a beer straight from the bottle for evaluative purposes. But if thats how the brewery intends it to be served, Im dr

0.04979205325508098
22oz '07 vintage picked up sometime last week at the Boise Coop for a very reasonable $4.. poured into my New Belgium snifter.		Fairly aggressive poure produced a creamy looking 3 finger light tan head atop a very clear amber brew with cherry juice highlites... bunches of leopard spot lacing.		Very assertive hop profile.. nose hair curling pine and citrus.. mildy floral... I know there are malts in there.. but wow.. This is my idea of a fresh American Barley Wine.		Palate is ultra smooth despite being pummeled by the hops.. Not as heavy body as I remember in previous years and I am okay with that... big clingy hop resins.. more pine and citrus rind.. not alot of yeast fruitiness..but it is identifiable... lower carbonation helps add to the body.. toasty and caramel malts really only start showing up as it warms... only passing glances of chocolate.		Another very good vintage of the Boardhead... and I need to pick up quite a few more for the cellar.. I think it will 

In [7]:
np.median(scores)

0.3495051530571942